<a href="https://colab.research.google.com/github/Io2007/simple-proxy/blob/master/nb/HuggingFace%20Course-Gemma3_(1B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center"><a href="https://huggingface.co/learn/nlp-course/en/chapter12/6?fw=pt"><img src="https://github.com/unslothai/notebooks/raw/main/assets/hf%20course.png" width="165"></a>
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

In this [Hugging Face](https://huggingface.co/learn/nlp-course/en/chapter12/6?fw=pt) and Unsloth notebook, you will learn to transform Gemma 3 (1B) into a Reasoning model using GRPO.

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Load up `Gemma 3 1B Instruct`, and set parameters

In [4]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-20 17:29:45 [__init__.py:256] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.17: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [5]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 10,           # Larger = higher accuracy, but might overfit
    lora_alpha = 10,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


### Data Prep
<a name="Data"></a>

We're using OpenAI's famous GSM8K dataset!

In [6]:
from datasets import load_dataset
dataset = load_dataset("lightblue/reasoning-multilingual-R1-Llama-70B-train",  split = "train")
dataset

README.md:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/9.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2477 [00:00<?, ? examples/s]

Dataset({
    features: ['en_prompt', 'dataset_name', 'language', 'translated_prompt_raw', 'num_tokens', 'translated_prompt', 'is_valid_prompt', 'row_id', 'res_finish_reason', 'response', 'is_valid_think', 'is_valid_res', 'think_lang', 'res_lang', 'think_fluency', 'prompt_think_lang_correct', 'answer_correctness', 'conversations'],
    num_rows: 2477
})

Let's look at the first row:

In [ ]:
dataset[0]["question"]

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [ ]:
dataset[0]["answer"]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

We notice all answers like about have a ####, so we extract it:

In [ ]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

'72'

We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:

In [7]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""Se le plantea un problema.
Piense en el problema y proporcione su solución.
Colóquelo entre {reasoning_start} y {reasoning_end}.
A continuación, proporcione su solución entre {solution_start}{solution_end}"""
system_prompt

'Se le plantea un problema.\nPiense en el problema y proporcione su solución.\nColóquelo entre <start_working_out> y <end_working_out>.\nA continuación, proporcione su solución entre <SOLUTION></SOLUTION>'

Let's map the dataset! and see the first row:

In [9]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["translated_prompt"]},
    ],
    "answer": x["response"],
})
dataset[0]

Map:   0%|          | 0/2477 [00:00<?, ? examples/s]

{'en_prompt': "I want to add files to an iso. The iso is large, and I'm on Linux.",
 'dataset_name': 'oasst2',
 'language': 'English',
 'translated_prompt_raw': "I want to add files to an iso. The iso is large, and I'm on Linux.",
 'num_tokens': -1,
 'translated_prompt': "I want to add files to an iso. The iso is large, and I'm on Linux.",
 'is_valid_prompt': True,
 'row_id': 0,
 'res_finish_reason': 'stop',
 'response': "<think>\nOkay, so I have this ISO file that I need to add some files to. The ISO is pretty large, and I'm using Linux. Alright, let me think about how to approach this. \n\nFirst, I know that an ISO file is basically a disk image, like a snapshot of a CD or DVD. So, if I want to add files to it, I probably need to modify that image. But how do I do that without messing up the existing content? I remember hearing about tools like `dd` for working with disk images, but I'm not sure if that's the right tool for adding files.\n\nMaybe I should look into tools that can mou

We create a regex format to match the reasoning sections and answers:

In [10]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [11]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [12]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [13]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [14]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

In [15]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [16]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [18]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 4e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 300,
    save_steps = 300,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,477 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 8,153,600/1,000,000,000 (0.82% trained)


******************** Question:
Assuming you divide a pizza into 8 equal slices and 3 people each eat 2 slices, how many slices remain? 
Answer:
<think>
Alright, let's try to figure out how many pizza slices are left after 3 people each eat 2 slices. Okay, so the pizza is divided into 8 equal slices. That means there are 8 slices in total. Now, each person eats 2 slices, and there are 3 people. Hmm, so I need to calculate the total number of slices eaten by all three people.

Let me think, if one person eats 2 slices, then three people would eat 2 slices multiplied by 3. So that's 2 times 3, which equals 6 slices in total. So, if 6 slices are eaten, how many are left from the original 8?

To find out how many slices remain, I need to subtract the number of slices eaten from the total number of slices. That would be 8 minus 6. Let me do that calculation: 8 minus 6 equals 2. So, there are 2 slices left after the three people have each eaten 2 slices.

Wait, let me double-check to make sur

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.000000,-1.000000,0.000000,93.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
2,-0.000000,-0.500000,1.000000,749.500000,0.000000,0.000000,-0.500000,0.000000,0.000000
3,0.000000,0.000000,0.000000,676.500000,0.000011,0.000000,0.000000,0.000000,0.000000
4,0.000000,-2.000000,0.000000,768.000000,0.000008,0.000000,-2.000000,0.000000,0.000000
5,0.000000,-1.000000,0.000000,768.000000,0.000010,0.000000,-1.000000,0.000000,0.000000
6,0.000000,-1.250000,0.500000,503.750000,0.000009,0.000000,-1.250000,0.000000,0.000000
7,0.000000,-1.250000,0.500000,130.250000,0.000007,0.000000,-1.250000,0.000000,0.000000
8,0.000000,-1.000000,0.000000,656.250000,0.000007,0.000000,-1.000000,0.000000,0.000000
9,0.000000,-1.250000,0.957427,768.000000,0.000010,0.000000,-1.250000,0.000000,0.000000
10,0.000000,-1.750000,0.500000,768.000000,0.000009,0.000000,-1.750000,0.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!
******************** Question:
Explica cómo la falacia ad hominem podría socavar la validez de un argumento en un debate sobre políticas medioambientales. 
Answer:
<think>
La falacia ad hominem es un tipo de error argumentativo en el que, en lugar de atacar o refutar los puntos o argumentos presentados por la otra parte, se critica o se ataca directamente a la persona que los presenta. En el contexto de un debate sobre políticas medioambientales, esta falacia podría socavar la validez de un argumento de varias maneras:

1. **Desviación del tema**: Al centrarse en el carácter, las intenciones o las acciones pasadas de la persona en lugar de los argumentos en sí, se desvía la atención del debate principal. Esto puede hacer que el público o los oponentes pierdan de vista los puntos clave de la discusión sobre las políticas medioambientales.

2. **Disminución de la credibilidad**: Si se ataca a la persona en lugar de sus argumentos, pue

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.000000,-1.000000,0.000000,93.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
2,-0.000000,-0.500000,1.000000,749.500000,0.000000,0.000000,-0.500000,0.000000,0.000000
3,0.000000,0.000000,0.000000,676.500000,0.000011,0.000000,0.000000,0.000000,0.000000
4,0.000000,-2.000000,0.000000,768.000000,0.000008,0.000000,-2.000000,0.000000,0.000000
5,0.000000,-1.000000,0.000000,768.000000,0.000010,0.000000,-1.000000,0.000000,0.000000
6,0.000000,-1.250000,0.500000,503.750000,0.000009,0.000000,-1.250000,0.000000,0.000000
7,0.000000,-1.250000,0.500000,130.250000,0.000007,0.000000,-1.250000,0.000000,0.000000
8,0.000000,-1.000000,0.000000,656.250000,0.000007,0.000000,-1.000000,0.000000,0.000000
9,0.000000,-1.250000,0.957427,768.000000,0.000010,0.000000,-1.250000,0.000000,0.000000
10,0.000000,-1.750000,0.500000,768.000000,0.000009,0.000000,-1.750000,0.000000,0.000000


******************** Question:
Interpretați implicațiile afirmației „orice număr întreg par mai mare decât 2 poate fi exprimat ca suma a două numere prime” și oferiți un exemplu pentru a o susține. 
Answer:
<think>
Pentru a interpreta implicațiile afirmației „orice număr întreg par mai mare decât 2 poate fi exprimat ca suma a două numere prime”, trebuie să înțelegem mai întâi ce înseamnă această afirmație. Ea sugerează că pentru orice număr par și mai mare decât 2, există două numere prime care, adunate împreună, sunt egale cu acel număr par.

Această afirmație este cunoscută sub numele de **Conjectura lui Goldbach**, una dintre cele mai faimoase conjecturi nerezolvate în teoria numerelor. Conjectura lui Goldbach a fost formulată pentru prima dată de matematicianul prusac Christian Goldbach în secolul al XVIII-lea și, deși a fost testată pentru foarte multe numere, încă nu există o dovadă matematică completă care să o confirme în general.

### Exemplu pentru a susține afirmația:
Să luă

<a name="Inference"></a>
### Inference
Now let's try the model we just trained!

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<bos><bos><start_of_turn>user
You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>

What is the sqrt of 101?<end_of_turn>
<start_of_turn>model
<start_working_out>
The square root of 101 is approximately 10.0498756.
<SOLUTION>
10.0498756<end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

('gemma-3/tokenizer_config.json',
 'gemma-3/special_tokens_map.json',
 'gemma-3/tokenizer.model',
 'gemma-3/added_tokens.json',
 'gemma-3/tokenizer.json')

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
